# Workstation Heartbeat

This example shows how in a separate thread from the Term the HeartBeat can be queried with Python. The HeartBeat is queried in a separate thread once a second.

The HeartBeat represents how many milliseconds it has been since the Term has received something from the Thales.

In [1]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode, ThalesRemoteScriptWrapper
import time
import threading

keepThreadRunning = True

# Watch Thread
The following function is used as a thread, in which the HearBeat is queried once a second. The HeartBeat time varies, for example, if EIS is measured at low frequencies, then this time is increased.

With getTermIsActive() it is checked with timeout whether the Term software still answers, since beside Thales also the Term could have crashed.

In [2]:
def watchThreadFunction():
    global keepThreadRunning

    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "Watch")

    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)

    while keepThreadRunning:
        time.sleep(1)
        active = zahnerZennium.getTermIsActive()
        print("active state: " + str(active))
        if active:
            print("beat count: " + str(zahnerZennium.getWorkstationHeartBeat()))
    zenniumConnection.disconnectFromTerm()
    return

# Main Program Sequence

The following is the main program flow, alongside which the Watch Thread runs. Here we simply measure something as an example.

In [3]:
if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "ScriptRemote")

    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()

    zahnerZennium.calibrateOffsets()

devel version


Now the watch thread is started in the main process, which runs alongside.

In [4]:
testThread = threading.Thread(target=watchThreadFunction)
testThread.start()
print("heartbeat thread started")

heartbeat thread started


In the following, an impedance spectrum is simply measured as an example.

In [5]:
zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
zahnerZennium.setAmplitude(10e-3)
zahnerZennium.setPotential(0)
zahnerZennium.setLowerFrequencyLimit(500)
zahnerZennium.setStartFrequency(1000)
zahnerZennium.setUpperFrequencyLimit(10000)
zahnerZennium.setLowerNumberOfPeriods(5)
zahnerZennium.setLowerStepsPerDecade(2)
zahnerZennium.setUpperNumberOfPeriods(20)
zahnerZennium.setUpperStepsPerDecade(20)
zahnerZennium.setScanDirection("startToMax")
zahnerZennium.setScanStrategy("single")

zahnerZennium.enablePotentiostat()

zahnerZennium.setFrequency(1)
zahnerZennium.setAmplitude(10e-3)
zahnerZennium.setNumberOfPeriods(3)

print("measurement start")
zahnerZennium.measureEIS()
print("measurement end")

zahnerZennium.disablePotentiostat()

measurement start
devel version
active state: True
beat count: 547.0
active state: True
beat count: 766.0
active state: True
beat count: 16.0
active state: True
beat count: 1032.0
active state: True
beat count: 140.0
active state: True
beat count: 1140.0
active state: True
beat count: 578.0
active state: True
beat count: 0.0
active state: True
beat count: 1000.0
active state: True
beat count: 500.0
active state: True
beat count: 32.0
active state: True
beat count: 1032.0
active state: True
beat count: 578.0
active state: True
beat count: 0.0
active state: True
beat count: 984.0
active state: True
beat count: 406.0
active state: True
beat count: 219.0
active state: True
beat count: 875.0
active state: True
beat count: 375.0
active state: True
beat count: 203.0
active state: True
beat count: 906.0
active state: True
beat count: 422.0
active state: True
beat count: 234.0
active state: True
beat count: 844.0
active state: True
beat count: 250.0
active state: True
beat count: 62.0
active st

'OK\r'

Close the watch thread and wait until it is closed.

In [6]:
print("thread kill")
keepThreadRunning = False
testThread.join()
print("thread killed")

thread kill
active state: True
beat count: 0.0
thread killed


# Disconnect

After the measurements are completed, the device switches back to the main potentiostat and the connection to the term is disconnected.

In [7]:
zenniumConnection.disconnectFromTerm()
print("finish")

finish
